In [1]:
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, Lasso, LinearRegression, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb
import lightgbm as lgb
import shap
from imblearn.pipeline import make_pipeline
from imblearn.pipeline import Pipeline
from imblearn import datasets
from imblearn.over_sampling import SMOTE
from tqdm import tqdm
from mne.decoding import SSD
from scipy import stats
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate, KFold, StratifiedKFold, cross_val_predict
from sklearn.metrics import recall_score, roc_auc_score, accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from skopt.utils import use_named_args
from skopt import gp_minimize
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report,confusion_matrix
from mne.filter import filter_data
import scipy.signal as sp_sig
import scipy.stats as sp_stats
import antropy as ant
from mne.filter import filter_data
from scipy.stats import zscore
from scipy.signal import wiener
from sklearn.linear_model import SGDRegressor, LinearRegression, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from yasa import sliding_window, bandpower_from_psd_ndarray
from skopt import BayesSearchCV
import time
import seaborn as sns
import sklearn
import pandas as pd
import numpy as np
import mne
import matplotlib.pyplot as plt
from sklearn.metrics import make_scorer
from sklearn.metrics import fbeta_score, make_scorer
from scipy.stats import pearsonr
import os

In [2]:
# Function to adjust values in an array
def adjust_values(arr):
    last_non_half = None  # Initialize a variable to keep track of the last non-0.5 value
    for i in range(len(arr)):  # Loop through each element in the array
        if arr[i] != 0.5:  # If the current element is not 0.5
            last_non_half = arr[i]  # Update the last non-0.5 value
        elif last_non_half is not None:  # If the current element is 0.5 and we have a last non-0.5 value
            arr[i] = last_non_half  # Set the current element to the last non-0.5 value
    return arr  # Return the adjusted array

# Function to compute the Pearson correlation coefficient between two arrays
def pearson_corr(y_true, y_pred):
    return pearsonr(y_true, y_pred)[0]  # [0] because pearsonr returns a tuple (correlation coefficient, p-value)

# Function to compute Source Signal Decomposition (SSD) component
def compute_ssd_com(raw, low_peak, stn_name):
    # Define the frequency range for the signal based on the low peak
    freqs_sig = low_peak-2, low_peak+2
    # Define the frequency range for the noise based on the low peak
    freqs_noise = low_peak-5, low_peak+5
    # Create a copy of the raw data and pick the specific channel (stn_name)
    raw_stn = raw.copy().pick(stn_name)
    # Initialize the SSD (Source Signal Decomposition) object
    ssd = SSD(
        info=raw_stn.info,  # Use the info from the raw_stn object
        reg="oas",  # Use Oracle Approximating Shrinkage for regularization
        sort_by_spectral_ratio=False,  # Disable sorting by spectral ratio for this example
        # Define filter parameters for the signal
        filt_params_signal=dict(
            l_freq=freqs_sig[0],  # Lower frequency bound
            h_freq=freqs_sig[1],  # Upper frequency bound
            l_trans_bandwidth=1,  # Lower transition bandwidth
            h_trans_bandwidth=1,  # Upper transition bandwidth
        ),
        # Define filter parameters for the noise
        filt_params_noise=dict(
            l_freq=freqs_noise[0],  # Lower frequency bound
            h_freq=freqs_noise[1],  # Upper frequency bound
            l_trans_bandwidth=1,  # Lower transition bandwidth
            h_trans_bandwidth=1,  # Upper transition bandwidth
        ),
    )
    # Fit the SSD model and transform the data to obtain SSD sources
    ssd_sources = ssd.fit_transform(X=raw_stn.get_data())
    # Select the first SSD component as the low-beta component
    lowbeta_com = ssd_sources[0]
    return lowbeta_com  # Return the low-beta component


In [3]:
# Get the current directory
current_path = os.getcwd()
data_path = current_path + '/fif_fileds_folder'

# Loop through all files in the current directory
for file in os.listdir(data_path):
    if file.endswith(".fif") and not file.endswith("_raw.fif"):
        # Create the new file name by appending '_raw' before the file extension
        new_file_name = file.replace(".fif", "_raw.fif")
        # Get the full file paths
        old_file_path = os.path.join(data_path, file)
        new_file_path = os.path.join(data_path, new_file_name)
        # Rename the file
        os.rename(old_file_path, new_file_path)
        print(f"Renamed '{file}' to '{new_file_name}'")

# Now, load the renamed .fif files
# List to hold the raw data
raw_data_list = []

# Loop through all files in the current directory
for file in os.listdir(data_path):
    if file.endswith("_raw.fif"):
        # Create the full file path
        file_path = os.path.join(data_path, file)
        print(f"Loading file: {file_path}")
        # Read the .fif file
        raw = mne.io.read_raw_fif(file_path, preload=True)
        # Append to the list
        raw_data_list.append(raw)

Loading file: /home/ben/EcoG_STN_Dev/fif_fileds_folder/FOG06_NS_sit_raw.fif
Opening raw data file /home/ben/EcoG_STN_Dev/fif_fileds_folder/FOG06_NS_sit_raw.fif...
    Range : 1250 ... 52337 =      5.000 ...   209.348 secs
Ready.
Reading 0 ... 51087  =      0.000 ...   204.348 secs...
Loading file: /home/ben/EcoG_STN_Dev/fif_fileds_folder/FOG16_ON_stand_raw.fif
Opening raw data file /home/ben/EcoG_STN_Dev/fif_fileds_folder/FOG16_ON_stand_raw.fif...
    Range : 1250 ... 42781 =      5.000 ...   171.124 secs
Ready.
Reading 0 ... 41531  =      0.000 ...   166.124 secs...
Loading file: /home/ben/EcoG_STN_Dev/fif_fileds_folder/FOG23_ON_stand_raw.fif
Opening raw data file /home/ben/EcoG_STN_Dev/fif_fileds_folder/FOG23_ON_stand_raw.fif...
    Range : 1250 ... 47389 =      5.000 ...   189.556 secs
Ready.
Reading 0 ... 46139  =      0.000 ...   184.556 secs...
Loading file: /home/ben/EcoG_STN_Dev/fif_fileds_folder/FOG20_ON_stand_raw.fif
Opening raw data file /home/ben/EcoG_STN_Dev/fif_fileds_fol

In [4]:
# Check the example raw data
raw_example = raw_data_list[10]
raw_example.info

Measurement date,"September 18, 2020 14:05:15 GMT"
Experimenter,Unknown
Participant,X
Digitized points,0 points
Good channels,19 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,250.00 Hz
Highpass,1.00 Hz
Lowpass,45.00 Hz


In [5]:
raw_example.ch_names #the names of the channels
#raw_example.plot()
# RS-EcoG LD&RD-STN

['RS12',
 'RS23',
 'RS34',
 'RS45',
 'RS56',
 'RS67',
 'RS78',
 'LD12',
 'LD23',
 'LD34',
 'LD13',
 'LD24',
 'LD14',
 'RD12',
 'RD23',
 'RD34',
 'RD13',
 'RD24',
 'RD14']

In [6]:
df = pd.read_excel(data_path+'/id_list.xlsx', index_col = [0])
print(len(df))
df = df[df['stn_lowbeta'] != 'no_signal'].reset_index(drop = True)
print(len(df))
df.head()

f_name = df['name'][0]
print(f_name)
# beijing = df[df['cohort'] == 'beijing'].reset_index(drop = True)

26
26
FOG01_NS_sit.fif


In [7]:
ex_folder = current_path+ '/preprocessing_data'
if not os.path.exists(ex_folder):
    os.makedirs(ex_folder)


for i in tqdm(range(len(raw_data_list))):
    f_name = df['name'][i]
    raw = raw_example
    stn_name = []
    for ch in raw.ch_names:
        if 'RD' in ch:
            stn_name += [ch]
    # remove bad channels
    for remove_ch in df.iloc[i]['removestnch'].split(','):
        for candi in stn_name:
            if remove_ch in candi:
                stn_name.remove(candi)
    # run ssd
    low_peak = df.iloc[i]['stn_lowbeta']
    if low_peak == 'no':
        low_peak = 16
    high_peak = df.iloc[i]['stn_highbeta']
    if high_peak == 'no':
        high_peak = 26
    lowbeta_com = compute_ssd_com(raw, low_peak, stn_name)
    highbeta_com = compute_ssd_com(raw, high_peak, stn_name)
    ecog_name = []
    for ch in raw.ch_names:
        if 'RS' in ch:
            ecog_name += [ch]

    # generate raw file
    sfreq=raw.info['sfreq']
    ch_names = ['low_beta', 'high_beta'] + ecog_name
    info = mne.create_info(ch_names=ch_names, sfreq=sfreq)
    raw_dc = mne.io.RawArray(np.vstack([np.stack([lowbeta_com, highbeta_com]), raw.get_data(ecog_name)]),info)
    # raw_dc = mne.io.RawArray(np.vstack([raw.get_data(stn_name), raw.get_data(ecog_name)]),info)
    # 1.5 s window, 1.5 Hz resolution
    window = 1.5
    overlap = 0
    step = window - overlap
    epochs = mne.make_fixed_length_epochs(raw_dc, duration=window, overlap = overlap, preload=False)
    # define frequency range
    channel = ['low_beta', 'high_beta'] 
    # generate features
    psd_all = mne.time_frequency.tfr_morlet (epochs, freqs = np.arange(4,44,1), n_cycles = (np.arange(4,44,1)/1.5).astype(int),
                                use_fft = True, 
                                return_itc = False, n_jobs = -1, picks = channel,average = False,
                                output = 'power')
    lowindex = (psd_all.freqs >= low_peak-1) & (psd_all.freqs <= low_peak+1)
    lowbeta_tar = psd_all.copy().pick('low_beta').data.squeeze().mean(axis = -1)[:, lowindex].mean(axis = -1)

    highindex = (psd_all.freqs >= high_peak-1) & (psd_all.freqs <= high_peak+1)
    highbeta_tar = psd_all.copy().pick('high_beta').data.squeeze().mean(axis = -1)[:, highindex].mean(axis = -1)
    stn_target = np.stack([lowbeta_tar,highbeta_tar]).T
    ex_name = f_name[:-4]+'_lowhighstn.npy'
    np.save('%s/%s'%(ex_folder, ex_name), stn_target)

    ecog_all = epochs.get_data(ecog_name)
    ex_name = f_name[:-4]+'_ecog.npy'
    np.save('%s/%s'%(ex_folder, ex_name), ecog_all)

    epochs_originial = mne.make_fixed_length_epochs(raw, duration=window, overlap = overlap, preload=False)
    stn_all = epochs_originial.get_data(stn_name)
    ex_name = f_name[:-4]+'_stn.npy'
    np.save('%s/%s'%(ex_folder, ex_name), stn_all)



  0%|          | 0/26 [00:00<?, ?it/s]

Setting up band-pass filter from 17 - 21 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 17.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 16.50 Hz)
- Upper passband edge: 21.00 Hz
- Upper transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 21.50 Hz)
- Filter length: 825 samples (3.300 s)

Setting up band-pass filter from 14 - 24 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 14.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Upper passband edge: 24.00 Hz
- Upper transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 24.50 Hz)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.


Using data from preloaded Raw for 199 events and 375 original time points ...
Not setting metadata
199 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 199 events and 375 original time points ...
0 bad epochs dropped


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.9s finished
  4%|▍         | 1/26 [00:01<00:29,  1.17s/it]

...

 96%|█████████▌| 25/26 [00:13<00:00,  2.88it/s]

Setting up band-pass filter from 15 - 19 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 15.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 14.50 Hz)
- Upper passband edge: 19.00 Hz
- Upper transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 19.50 Hz)
- Filter length: 825 samples (3.300 s)

Setting up band-pass filter from 12 - 22 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 12.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 11.50 Hz)
- Upper passband edge: 22.00 Hz
- Upper transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 22.50 Hz)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.2s finished


Using data from preloaded Raw for 199 events and 375 original time points ...
Not setting metadata
199 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 199 events and 375 original time points ...
0 bad epochs dropped


100%|██████████| 26/26 [00:14<00:00,  1.84it/s]


In [8]:
data_ecog = np.load(ex_folder + '/' + 'FOG01_NS_sit_ecog.npy')
data_lowhigh = np.load(ex_folder + '/' + 'FOG01_NS_sit_lowhighstn.npy')
data_stn = np.load(ex_folder + '/' + 'FOG01_NS_sit_stn.npy')
print(data_ecog.shape)
print(data_lowhigh.shape)
print(data_stn.shape)
# 250*1.5 =375
# 300 / 1.5 =200

print(ecog_name)
print(stn_name)
print(data_stn[0])

(199, 7, 375)
(199, 2)
(199, 6, 375)
['RS12', 'RS23', 'RS34', 'RS45', 'RS56', 'RS67', 'RS78']
['RD23', 'RD34', 'RD24']
[[-3.19077287e-07  9.07073684e-07  1.73102444e-06 ...  5.86486863e-07
   4.55931456e-07  2.64621292e-07]
 [ 3.09396206e-08 -1.50894465e-07 -5.07722234e-07 ... -5.26242161e-07
  -6.76469483e-07 -5.71871283e-07]
 [ 4.29538460e-07 -1.17773607e-06 -2.43963518e-06 ... -2.65257836e-06
  -2.92655045e-06 -2.66119764e-06]
 [-2.88138153e-07  7.56180611e-07  1.22330482e-06 ...  6.02446164e-08
  -2.20538013e-07 -3.07249962e-07]
 [ 4.60478049e-07 -1.32863045e-06 -2.94735719e-06 ... -3.17882063e-06
  -3.60302010e-06 -3.23306904e-06]
 [ 1.41400790e-07 -4.21556791e-07 -1.21633298e-06 ... -2.59233389e-06
  -3.14708836e-06 -2.96844746e-06]]


In [9]:
training_data_folder = 'model_training_data'
train_set = 'model_training_data/train_set'
validation_set = 'model_training_data/validation_set'
test_set = 'model_training_data/test_set'

if not os.path.exists(training_data_folder):
    os.makedirs(training_data_folder)
if not os.path.exists(train_set):
    os.makedirs(train_set)
if not os.path.exists(validation_set):
    os.makedirs(validation_set)
if not os.path.exists(test_set):
    os.makedirs(test_set)

In [10]:
from sklearn.model_selection import train_test_split
# Data Separation
data_ecog = np.load(ex_folder + '/' + 'FOG01_NS_sit_ecog.npy')
data_lowhigh = np.load(ex_folder + '/' + 'FOG01_NS_sit_lowhighstn.npy')
data_stn = np.load(ex_folder + '/' + 'FOG01_NS_sit_stn.npy')

# Ensure the output directories exist
train_dir = 'model_training_data/train_set'
val_dir = 'model_training_data/validation_set'
test_dir = 'model_training_data/test_set'

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Function to save data pieces
def save_data_piece(data_piece, label_piece, directory, index):
    np.save(os.path.join(directory, f'ecog_{index}.npy'), data_piece)
    np.save(os.path.join(directory, f'stn_{index}.npy'), label_piece)

# Split the data into 199 pieces
indices = np.arange(199)
train_indices, temp_indices = train_test_split(indices, test_size=0.2, random_state=42)
val_indices, test_indices = train_test_split(temp_indices, test_size=0.5, random_state=42)

# Save the data pieces into the corresponding directories
for idx in train_indices:
    save_data_piece(data_ecog[idx], data_stn[idx], train_dir, idx)

for idx in val_indices:
    save_data_piece(data_ecog[idx], data_stn[idx], val_dir, idx)

for idx in test_indices:
    save_data_piece(data_ecog[idx], data_stn[idx], test_dir, idx)

print(f'Training set: {len(train_indices)} pieces')
print(f'Validation set: {len(val_indices)} pieces')
print(f'Test set: {len(test_indices)} pieces')


Training set: 159 pieces
Validation set: 20 pieces
Test set: 20 pieces


In [11]:
traning_data_example = np.load(current_path + '/' + train_dir +'/ecog_0.npy') 
label_data_example = np.load(current_path + '/' + train_dir +'/stn_0.npy') 
print(traning_data_example.shape)
print(traning_data_example)
print(label_data_example.shape)
print(label_data_example)

max_data = np.max(traning_data_example)
min_data = np.min(traning_data_example)
max_label = np.max(label_data_example)
min_label = np.min(label_data_example)
print(max_label)
print(min_label)

(7, 375)
[[-3.57784813e-07  1.20237257e-06  2.16530020e-06 ...  3.11021322e-05
   3.12111260e-05  2.80098393e-05]
 [-5.40969552e-07  9.03791715e-07  7.93941240e-07 ...  8.44373062e-06
   3.35157870e-06 -1.33591584e-07]
 [-4.10103702e-08  7.93561483e-08 -4.91468448e-08 ...  8.31823399e-07
   1.13425550e-07 -5.39971552e-07]
 ...
 [-7.26199232e-07  2.40081249e-06  9.77178161e-07 ...  2.95666678e-06
  -2.81858456e-06 -7.73639204e-06]
 [ 6.62694538e-06 -2.07122794e-05 -4.45263322e-05 ...  2.47650823e-05
   3.29564464e-05  3.77390643e-05]
 [-3.13919509e-06  1.00203761e-05  2.37436016e-05 ... -1.98272010e-05
  -1.18511934e-05  5.47560194e-06]]
(6, 375)
[[-3.19077287e-07  9.07073684e-07  1.73102444e-06 ...  5.86486863e-07
   4.55931456e-07  2.64621292e-07]
 [ 3.09396206e-08 -1.50894465e-07 -5.07722234e-07 ... -5.26242161e-07
  -6.76469483e-07 -5.71871283e-07]
 [ 4.29538460e-07 -1.17773607e-06 -2.43963518e-06 ... -2.65257836e-06
  -2.92655045e-06 -2.66119764e-06]
 [-2.88138153e-07  7.56180611e-

In [12]:
# Data Augmentation
# Function to add random Gaussian noise with automatically calculated noise factor
def add_noise(data, label, noise_level=0.3):
    # Calculate the extreme values of data and label
    data_range = np.max(data) - np.min(data)
    label_range = np.max(label) - np.min(label)

    # Calculate noise factor
    noise_factor_data = (data_range)*noise_level
    noise_factor_label = (label_range)*noise_level
    
    # Add noise to data
    noise = np.random.normal(0, noise_factor_data, data.shape[1])
    noise_data = data + noise
    
    # Add noise to label
    noise_label = noise*(noise_factor_label/noise_factor_data)
    noise_label = label + noise_label

    return noise_data, noise_label

# Function to scale data
def scale_data(data, label, scaling_factor=0.2):
    scale = 1 + np.random.uniform(-scaling_factor, scaling_factor)
    scale_data = data * scale
    scale_label = label * scale
    return scale_data, scale_label

def shift_data(data, label, shift_fraction=0.1):
    data_range = np.max(data) - np.min(data)
    label_range = np.max(label) - np.min(label)
    shift_max = shift_fraction * data_range
    data_shift = np.random.uniform(-shift_max, shift_max)
    shifted_data = data + data_shift
    shifted_label = label + (data_shift * label_range / data_range)
    return shifted_data, shifted_label

# Function to flip data
def flip_data(data, label):
    flip_data = np.flip(data,1)
    flip_label = np.flip(label,1)
    return flip_data, flip_label

# Function to augment a single sample
def augment_sample(data, label):
    num_augmentations = 10
    augmented_samples = []
    
    for _ in range(num_augmentations):
        augmented_data = np.copy(data)
        augmented_label = np.copy(label)
        
        augmented_data, augmented_label = add_noise(augmented_data, augmented_label)
        augmented_data, augmented_label = scale_data(augmented_data, augmented_label)
        augmented_data, augmented_label = shift_data(augmented_data, augmented_label)
        # if np.random.rand() > 0.5:
        #     augmented_data, augmented_label = flip_data(augmented_data, augmented_label)
        
        augmented_samples.append((augmented_data, augmented_label))
    
    return augmented_samples

# Load the data
train_dir = 'model_training_data/train_set'
train_data_files = sorted([f for f in os.listdir(train_dir) if f.startswith('ecog_')])
train_label_files = sorted([f for f in os.listdir(train_dir) if f.startswith('stn_')])

# Create augmented directories
aug_train_dir = 'model_training_data/augmented_train_set'
os.makedirs(aug_train_dir, exist_ok=True)

# Process each training sample
augmented_index = 0
for data_file, label_file in zip(train_data_files, train_label_files):
    data_path = os.path.join(train_dir, data_file)
    label_path = os.path.join(train_dir, label_file)
    data = np.load(data_path)
    label = np.load(label_path)
    
    augmented_samples = augment_sample(data, label)
    
    for aug_data, aug_label in augmented_samples:
        np.save(os.path.join(aug_train_dir, f'aug_ecog_{augmented_index}.npy'), aug_data)
        np.save(os.path.join(aug_train_dir, f'aug_stn_{augmented_index}.npy'), aug_label)
        augmented_index += 1

print(f'Augmented training set: {augmented_index} pieces')
print(aug_data.shape)
print(aug_label.shape)


Augmented training set: 1590 pieces
(7, 375)
(6, 375)


In [13]:
# Function to pad data to make it 32-divisible
def padding(data):
    pad_height = (32 - data.shape[0] % 32) % 32
    pad_width = (32 - data.shape[1] % 32) % 32
    padded_data = np.pad(data, ((0, pad_height), (0, pad_width)), mode='constant')
    return padded_data

# Load the augmented data
aug_train_dir = 'model_training_data/augmented_train_set'
padded_train_dir = 'model_training_data/padded_train_set'
aug_nopad_train_dir = 'model_training_data/aug_nopad_train_set'
os.makedirs(padded_train_dir, exist_ok=True)
os.makedirs(aug_nopad_train_dir, exist_ok=True)

# Get files for ecog and stn
ecog_files = sorted([f for f in os.listdir(aug_train_dir) if f.startswith('aug_ecog_')])
stn_files = sorted([f for f in os.listdir(aug_train_dir) if f.startswith('aug_stn_')])

# Debugging: Check if files exist
print(f'Found {len(ecog_files)} ecog files')
print(f'Found {len(stn_files)} stn files')

# Process each augmented sample for padding
padded_index = 0
for ecog_file, stn_file in zip(ecog_files, stn_files):
    ecog_path = os.path.join(aug_train_dir, ecog_file)
    stn_path = os.path.join(aug_train_dir, stn_file)
    
    ecog_data = np.load(ecog_path)
    stn_data = np.load(stn_path)
    
    # Combine data
    combined_data = np.vstack((ecog_data, stn_data))
    conditional_label = ecog_data
    
    padded_combined_data = padding(combined_data)
    padded_conditional_label = padding(conditional_label)
    
    np.save(os.path.join(padded_train_dir, f'padded_combined_data_{padded_index}.npy'), padded_combined_data)
    np.save(os.path.join(padded_train_dir, f'padded_conditional_label_{padded_index}.npy'), padded_conditional_label)

    np.save(os.path.join(aug_nopad_train_dir, f'combined_data_{padded_index}.npy'), combined_data)
    np.save(os.path.join(aug_nopad_train_dir, f'conditional_label_{padded_index}.npy'), conditional_label)
    padded_index += 1

print(combined_data.shape)
print(conditional_label.shape)

print(f'Padded training set: {padded_index} pieces')
print(f'Augmented non Padding training set: {padded_index} pieces')

Found 1590 ecog files
Found 1590 stn files
(13, 375)
(7, 375)
Padded training set: 1590 pieces
Augmented non Padding training set: 1590 pieces


In [14]:
padded_data_example = np.load(current_path + '/' + padded_train_dir +'/padded_combined_data_0.npy') 
padded_label_example = np.load(current_path + '/' + padded_train_dir +'/padded_conditional_label_0.npy') 

aug_nopad_data_example = np.load(current_path + '/' + aug_nopad_train_dir  +'/combined_data_0.npy') 
aug_nopad_label_example = np.load(current_path + '/' + aug_nopad_train_dir  +'/conditional_label_0.npy') 

print(padded_data_example.shape)
print(padded_label_example.shape)

print(aug_nopad_data_example.shape)
print(aug_nopad_label_example.shape)

row_number_1 = 6
print(np.array_equal(padded_combined_data[7:13,:375], stn_data))
row_number_2 = 7
print(np.array_equal(padded_data_example[row_number_2,:], padded_label_example[row_number_2,:]))
print(np.array_equal(aug_nopad_data_example[:7,:], aug_nopad_label_example))

(32, 384)
(32, 384)
(13, 375)
(7, 375)
True
False
True
